In [ ]:
import paddle
import paddle.fluid as fluid
import numpy as np
import os
import matplotlib.pyplot as plt

BUF_SIZE = 500
BATCH_SIZE = 50


#读取数据集
train_reader = paddle.batch(
    paddle.reader.shuffle(paddle.dataset.uci_housing.train(),
                          buf_size=BUF_SIZE),
                          batch_size=BATCH_SIZE)
test_reader = paddle.batch(
    paddle.reader.shuffle(paddle.dataset.uci_housing.test(),
                          buf_size=BUF_SIZE),
                          batch_size=BATCH_SIZE
)

In [ ]:
#查看uci_housing数据
train_data = paddle.dataset.uci_housing.train()
sampledata = next(train_data())
print(sampledata)

配置网络

In [ ]:
x = fluid.layers.data(name='x', shape=[13], dtype='float32')
y = fluid.layers.data(name='y',shape=[1],dtype='float32')
y_predict = fluid.layers.fc(input=x,size=1,act=None)

#定义损失函数
cost = fluid.layers.square_error_cost(input=y_predict,label=y)
avg_cost = fluid.layers_mean(cost)

#定义优化器
optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.01)
opts = optimizer.minimize(avg_cost)
test_program = fluid.default_main_program().clone(for_test=True)

在上述模型配置完毕后，得到两个fluid.Program：fluid.default_startup_program() 与fluid.default_main_program() 。

参数初始化操作会被写入fluid.default_startup_program()

fluid.default_main_program()用于获取默认或全局main program(主程序)。该主程序用于训练和测试模型。fluid.layers 中的所有layer函数可以向 default_main_program 中添加算子和变量。default_main_program 是fluid的许多编程接口（API）的Program参数的缺省值。例如,当用户program没有传入的时候，Executor.run()会默认执行 default_main_program 。

模型训练和模型评估

In [ ]:
#创建Executor
#定义运算场所是CPU还是GPU
#Executor：接受传入的program，在通过run()的方法运行program
use_cuda = False
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
exe = fluid.Executor(place)
exe.run(fluid.default_startup_program())

#定义输入数据的维度
feeder = fluid.DataFeeder(place=place,feed_list=[x,y])#feed_list：向模型输入的变量表或变量表名


#绘制训练过程损失值变化趋势
iter = 0
iters = []
train_costs = []


def draw_train_process(iters,train_costs):
    title = "training cost"
    plt.title(title,fontsize=24)
    plt.xlabel("iter",fontsize=14)
    plt.ylabel("cost",fontsize=14)
    plt.plot(iters,train_costs,color='red',label='trian cost')
    plt.grid()
    plt.show()

#训练并保存模型

epoch = 50
model_save_dir = "/model/fit_a_line.inference.model"

for e in range(epoch):

    train_cost = 0
    for batch_id,data in enumerate(train_reader()):
        train_cost = exe.run(program=fluid.default_main_program(),
                             feed=feeder.feed(data),
                             fetch_list=[avg_cost])
        if batch_id % 40 == 0:
            print("Epoch:%d,cost:%0.5f"%(e,train_cost[0][0]))
        
        iter = iter + BATCH_SIZE
        iters.append(iter)
        train_costs.append(train_cost[0][0])

    test_cost = 0
    for batch_id,data in enumerate(test_reader()):
        test_cost = exe.run(program=fluid.test_program(),
                             feed=feeder.feed(data),
                             fetch_list=[avg_cost])
        if batch_id % 40 == 0:
            print("Epoch:%d,cost:%0.5f"%(e,test_cost[0][0]))


if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)

print("save model to %s"%(model_save_dir))

#保存训练参数到指定路径中，构建一个专门用预测的program
fluid.io.save_inference_model(model_save_dir,   #保存推理model的路径
                                  ['x'],            #推理（inference）需要 feed 的数据
                                  [y_predict],      #保存推理（inference）结果的 Variables
                                  exe)              #exe 保存 inference model
draw_train_process(iters,train_costs)




模型预测

In [ ]:
#创建预测用的Executor

infer_exe = fluid.Executor(place)    #创建推测用的executor
inference_scope = fluid.core.Scope() #Scope指定作用域

infer_results=[]
groud_truths=[]

#绘制真实值和预测值对比图
def draw_infer_result(groud_truths,infer_results):
    title='Boston'
    plt.title(title, fontsize=24)
    x = np.arange(1,20) 
    y = x
    plt.plot(x, y)
    plt.xlabel('ground truth', fontsize=14)
    plt.ylabel('infer result', fontsize=14)
    plt.scatter(groud_truths, infer_results,color='green',label='training cost') 
    plt.grid()
    plt.show()


with fluid.scope_guard(inference_scope):#修改全局/默认作用域（scope）, 运行时中的所有变量都将分配给新的scope。
    #从指定目录中加载 推理model(inference model)
    [inference_program,                             #推理的program
     feed_target_names,                             #需要在推理program中提供数据的变量名称
     fetch_targets] = fluid.io.load_inference_model(#fetch_targets: 推断结果
                                    model_save_dir, #model_save_dir:模型训练路径 
                                    infer_exe)      #infer_exe: 预测用executor
    #获取预测数据
    infer_reader = paddle.batch(paddle.dataset.uci_housing.test(),  #获取uci_housing的测试数据
                          batch_size=200)                           #从测试数据中读取一个大小为200的batch数据
    #从test_reader中分割x
    test_data = next(infer_reader())
    test_x = np.array([data[0] for data in test_data]).astype("float32")
    test_y= np.array([data[1] for data in test_data]).astype("float32")
    results = infer_exe.run(inference_program,                              #预测模型
                            feed={feed_target_names[0]: np.array(test_x)},  #喂入要预测的x值
                            fetch_list=fetch_targets)                       #得到推测结果 
                            
    print("infer results: (House Price)")
    for idx, val in enumerate(results[0]):
        print("%d: %.2f" % (idx, val))
        infer_results.append(val)
    print("ground truth:")
    for idx, val in enumerate(test_y):
        print("%d: %.2f" % (idx, val))
        groud_truths.append(val)
    draw_infer_result(groud_truths,infer_results)